<a href="https://colab.research.google.com/github/IshtarMM/Practical_Course/blob/master/Copy_of_Answers_MicrobiomeData_KemenLab_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##<font color='red'>Q1:</font>

In [ ]:
table(metadata$Year, metadata$Season, metadata$Infection_Stage)

##<font color='red'>Q2:</font>

In [ ]:
print(paste("Maximum Numer of Reads per Sample: ", max(tab_sample$ReadSumPerSample)))
print(paste("Minimum Numer of Reads per Sample: ", min(tab_sample$ReadSumPerSample)))
print(paste("Average Numer of Reads per Sample: ", mean(tab_sample$ReadSumPerSample)))

##<font color='red'>Q3:</font>

In [ ]:
cat("Maximum value:", max(tab_taxa1$Abundance), "Corresponding ID:", tab_taxa1$ID[which.max(tab_taxa1$Abundance)], "Corresponding Genus:", tab_taxa1$Genus[which.max(tab_taxa1$Abundance)], "\n")

# Find the maximum abundance and corresponding OTU for each ID
library(dplyr)

max_abundance <- aggregate(Abundance ~ ID, data = tab_taxa1, FUN = max)
max_otu <- aggregate(OTU ~ ID, data = tab_taxa1, FUN = function(x) x[which.max(tab_taxa1$Abundance)])

abs_abundance <- tab_taxa1 %>%
  group_by(ID) %>%
  slice_max(Abundance) %>%
  ungroup()
abs_abundance <- abs_abundance %>% select(ID, OTU_Genus, Abundance)
abs_abundance <- abs_abundance %>% arrange(desc(Abundance))

print(abs_abundance)

##<font color='red'>Q4:</font>

In [ ]:
#First we create "tab_taxa1" again but for the "Family" level
ra_tab <- decostand(data[-c(1)], method="total", MARGIN=1)
ra_tab$ID <- rownames(ra_tab)
melt_tab <- melt(ra_tab)
colnames(melt_tab) = c("ID","OTU","RelativeAbundance")
tab_taxa <- merge(x =melt_tab , y = taxonomy , by.x  = "OTU" , by.y = "OTU")
agg_tab <- aggregate(tab_taxa[,3], list(tab_taxa$Family), sum)
colnames(agg_tab) <- c("Family","SumAbundance")
agg_tab$Family1 <- agg_tab$Family
agg_tab$Family1[agg_tab$SumAbundance <= 20] <- "Other"
tab_taxa1 <- merge(tab_taxa, agg_tab ,by ="Family")
tab_taxa1$Family1 <- reorder(tab_taxa1$Family1 , tab_taxa1$SumAbundance)
tab_taxa1 <- merge(x =tab_taxa1  , y = metadata, by = "ID")
tab_taxa1$Year = as.factor(tab_taxa1$Year)

In [ ]:
#Next, we can create the plot
p5 <- ggplot() + geom_bar(data=tab_taxa1, aes(x=Infection_Stage, y=RelativeAbundance, fill=Family1), stat="identity", position="fill") + scale_color_brewer(palette="Dark2")+
  theme_bw() + facet_wrap(.~Year, scales="free_x", nrow=1, strip.position = "bottom") +
  theme(text = element_text(size = 19 , colour = "black"),
                       axis.text = element_text(size = 19 , colour = "black" ),
                       axis.title = element_text(size = 19 , colour = "black" ),
                       strip.text = element_text(size = 19 , colour = "black" ),
                       axis.text.x = element_text(angle = 60, hjust = 1 , colour="black", size = 19),
                       panel.grid.major = element_blank(),
                       panel.grid.minor = element_blank(),
                       axis.line = element_line(colour = "black") ,
                       legend.title = element_text(color = "black", size = 19),
                       legend.text = element_text(color = "black", size = 19) ,
                       strip.background = element_rect(colour = "black", fill = "white"))

options(repr.plot.width=15, repr.plot.height=8)
p5

In [ ]:
anova <- aov(RelativeAbundance ~ Infection_Stage, data = tab_taxa1)
summary(anova)
print(paste("2014:", TukeyHSD(aov(RelativeAbundance ~ Infection_Stage, data = subset(tab_taxa1, Year %in% c("2014"))))))
print(paste("2015:", TukeyHSD(aov(RelativeAbundance ~ Infection_Stage, data = subset(tab_taxa1, Year %in% c("2015"))))))
print(paste("2016:", TukeyHSD(aov(RelativeAbundance ~ Infection_Stage, data = subset(tab_taxa1, Year %in% c("2016"))))))
print(paste("2017:", TukeyHSD(aov(RelativeAbundance ~ Infection_Stage, data = subset(tab_taxa1, Year %in% c("2017"))))))
print(paste("2018:", TukeyHSD(aov(RelativeAbundance ~ Infection_Stage, data = subset(tab_taxa1, Year %in% c("2018"))))))
print(paste("2019:", TukeyHSD(aov(RelativeAbundance ~ Infection_Stage, data = subset(tab_taxa1, Year %in% c("2019"))))))

##<font color='red'>Q5:</font>

In [ ]:
plot_alpha_diversity_infection <- ggplot(alpha_div, aes(x=Infection_Stage, y=ShannonDiversity)) + geom_boxplot(fill="#009E73", color="black", outlier.shape=NA, width=0.2, lwd=0.4, linetype="dashed", position=position_dodge(width=6)) +
    stat_boxplot(aes(ymin = ..lower.., ymax = ..upper..) ,fill='#009E73' ,outlier.shape=NA ,width=0.7 , lwd = 0.4) +
    stat_boxplot(geom = "errorbar", aes(ymin = ..ymax..), width=0.3) +
    stat_boxplot(geom = "errorbar", aes(ymax = ..ymin..) ,width=0.3) +
    geom_jitter(alpha = 1 , color = "black" , position = position_jitter(width = 0.3) , size = 0.2) +
    theme(axis.text.x = element_text(angle = 60, hjust = 1 , colour="black", size = 14),
          axis.text.y = element_text(angle=90, hjust=1 , size = 14 , colour="black" ) ,
          axis.title.y =  element_text(angle=90, size = 14) ,
          axis.title.x = element_blank() ,
          text = element_text(size = 40),
          panel.background = element_rect(colour = "black" , fill = "NA"),
          panel.grid.major = element_blank(), panel.grid.minor = element_blank()) + ylim(0,6) #+
    #stat_compare_means(aes(label=paste0("p = ", ..p.signif..)), method="wilcox.test")

options(repr.plot.width=8, repr.plot.height=10)
plot_alpha_diversity_infection

wilcox.test(ShannonDiversity ~ Infection_Stage, data = alpha_div)

##<font color='red'>Q6:</font>

In [ ]:
metadata$Comp_Inf <- paste(metadata$Compartment, metadata$Infection_Stage, sep="_")

pca_bray_rarefied.tab <- dudi.pco(vegdist(data[-c(1)], method = "bray", binary=FALSE), scannf=F)
metadata$Comp_Inf <- as.factor(metadata$Comp_Inf)
options(repr.plot.width=8, repr.plot.height=7)
s.class(pca_bray_rarefied.tab$li, metadata$Comp_Inf, xax=1, yax=2, cpoint=3, grid=F, addaxes=T, cellipse=1, cstar=0, axesell=0, col = c("blue","red", "black", "purple"), clabel = 2, sub="Bray-Curtis")

aov <- adonis2(data[-c(1)] ~ metadata$Comp_Inf, permutations = 99, method = "bray")
aov

##<font color='red'>Q7:</font>

In [ ]:
#Hint: first subset the otu table of infected samples
metadata_infected_samples <- subset(metadata , metadata$Infection_Stage=="Infected")
data1 <- data[rownames(data)%in% rownames(metadata_infected_samples), ]
data_net <- data.frame(t(data1[-c(1)]))

In [ ]:
data_net$OTU <- rownames(data_net)
data_net <- merge(taxonomy[c("OTU" ,"Genus")] , data_net , by = "OTU")
data_net$OTU_Genus <- paste(data_net$OTU , data_net$Genus , sep = "_")
rownames(data_net) <- data_net$OTU_Genus
data_net <- subset(data_net , select = -c(OTU , Genus , OTU_Genus))
data_net <- data_net[rowSums(data_net)>=2000 , ]

In [ ]:
sparcc.data <- sparcc(t(data_net))
colnames(sparcc.data$Cor) <- rownames(data_net)
sparcc.graph <- abs(sparcc.data$Cor) >= 0.3
diag(sparcc.graph) <- 0
#sparcc.graph = sparcc.graph[rowSums(sparcc.graph)>=4,]
#sparcc.graph = sparcc.graph[,(colnames(sparcc.graph) %in% rownames(sparcc.graph))]
library(Matrix)
sparcc.graph <- Matrix(sparcc.graph, sparse=TRUE)
ig.sparcc <- adj2igraph(sparcc.graph,vertex.attr = list(name = rownames(sparcc.graph), ff = 1:nrow(sparcc.graph)))

In [ ]:
options(repr.plot.width=20, repr.plot.height=20)
cor_values <- sparcc.data$Cor[which(E(ig.sparcc) != 0)]
edge_colors <- ifelse(cor_values >= 0, "orange", "gray")
plot(ig.sparcc, vertex.label = colnames(sparcc.graph), main = "SparCC", vertex.size = 3, edge.color = edge_colors, edge.width = 1, edge.curved=0.5)

In [ ]:
library(igraph)
closeness <- data.frame(closeness(ig.sparcc, mode="all", weights=NA))
betweenness <- data.frame(betweenness(ig.sparcc, directed=F, normalized=TRUE))
degree <- data.frame(degree(ig.sparcc), mode="all")
net_analysis <- cbind(closeness, betweenness[1], degree)
colnames(net_analysis) <- c("ClosenessCentrality", "BetweennessCentrality", "Degree")

In [ ]:
taxonomy$OTU_Genus <- paste(taxonomy$OTU, taxonomy$Genus, sep="_")
net_analysis <- merge(net_analysis, taxonomy, by.x="row.names", by.y="OTU_Genus")
colnames(net_analysis)[1] <- "OTU_Genus"
net_analysis <- na.omit(net_analysis)
net_analysis <- net_analysis[!(net_analysis$ClosenessCentrality==1 & net_analysis$BetweennessCentrality==0), ]

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)
dd.sparcc <- degree.distribution(ig.sparcc)
plot(0:(length(dd.sparcc)-1), dd.sparcc, type="b",
      ylab="Frequency", xlab="Degree", main="Degree Distribution", col="forestgreen")
legend("topright", legend="Degree Distribution", col="forestgreen", pch=1, lty=1)

In [ ]:
n <- 10
library(ggrepel)
top5_BC <- net_analysis[net_analysis$BetweennessCentrality > quantile(net_analysis$BetweennessCentrality, prob=1-n/100),]$OTU_Genus
top5_BC_min <- min(net_analysis[net_analysis$BetweennessCentrality > quantile(net_analysis$BetweennessCentrality, prob=1-n/100),]$BetweennessCentrality)
top5_CC <- net_analysis[net_analysis$ClosenessCentrality > quantile(net_analysis$ClosenessCentrality, prob=1-n/100),]$OTU_Genus
top5_CC_min <- min(net_analysis[net_analysis$ClosenessCentrality > quantile(net_analysis$ClosenessCentrality, prob=1-n/100),]$ClosenessCentrality)

In [ ]:
p5_Intersection <- top5_BC[which((top5_BC %in% top5_CC) == TRUE)]

p5_IntersectionH = p5_Intersection
length(p5_IntersectionH)

In [ ]:
core = as.data.frame(p5_IntersectionH)
core$core = "Yes"
colnames(core) = c("Species", "HUB")
select = subset(net_analysis, !(OTU_Genus %in% core$Species))
select$core = "No"
select1 = select[,c("OTU_Genus", "core")]
colnames(select1) = c("Species", "HUB")
newnet_analysis = rbind(select1, core)
newnet_analysisH = merge(x=net_analysis, y=newnet_analysis, by.x="OTU_Genus", by.y="Species")
newnet_analysisH <- newnet_analysisH[complete.cases(newnet_analysisH),]
newnet_analysisH = newnet_analysisH[newnet_analysisH$Degree > 10, ]
head(newnet_analysisH)

In [ ]:
expression <- ggplot(newnet_analysisH, aes(x=ClosenessCentrality, y=BetweennessCentrality, label=Genus))
z <- expression + geom_point(aes(colour=Degree, size = Degree) ,stat= "identity",position="identity",alpha= 0.4 ) +
    scale_colour_gradient(low = "blue", high = "red") +
    labs(title = "Hub analysis") +
    scale_size_area(max_size = 10)+
    geom_vline(xintercept = top5_CC_min, linetype="dashed") +
    geom_hline(yintercept = top5_BC_min, linetype= "dashed") +
    scale_x_continuous("Closeness Centrality") +
    scale_y_continuous("Betweenness Centrality") +
    scale_shape_manual(values=c(21, 19)) +
    geom_text_repel(data=subset(newnet_analysisH, HUB=="Yes"),size = 7,
                    box.padding   = 1.5,
                    point.padding = 0.5,
                    force         = 100,
                    segment.size  = 0.4,
                    segment.color = "darkgreen") +
    theme_bw()+theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank()) +
    theme(text = element_text(size=15))
    options(repr.plot.width=10, repr.plot.height=7)
z